# KoBART(추출요약)

In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip install torchmetrics

In [ ]:
!pip install kss

In [ ]:
import math
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim

from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

from torch.nn.init import xavier_uniform_

import pytorch_lightning as pl
import torchmetrics
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import kss

In [ ]:
MAX_TOKEN_COUNT = 512
N_EPOCHS = 3
BATCH_SIZE = 2

https://www.exxactcorp.com/blog/Deep-Learning/advanced-pytorch-lightning-using-torchmetrics-and-lightning-flash

In [ ]:
RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


42

## 데이터

In [ ]:
# 학습
DATA_TRAIN_PATH = '/content/drive/MyDrive/데이터/금융 요약/train_original.json'
df = pd.read_json(DATA_TRAIN_PATH)
df = df.dropna()
len(df)#, len(val_df)

243983

In [ ]:
# 검증
DATA_TEST_PATH = '/content/drive/MyDrive/데이터/금융 요약/valid_original.json'
test_df = pd.read_json(DATA_TEST_PATH)
test_df = test_df.dropna()
len(test_df)

30122

In [ ]:
# 학습데이터에서 트레인/테스트 분리
train_df, val_df = train_test_split(df, test_size=0.05)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
train_df.shape, val_df.shape, test_df.shape

((231783, 3), (12200, 3), (30122, 3))

In [ ]:
# 데이터 사이즈 줄이기
downsize = 10000
train_df = train_df[:downsize]
test_df = test_df[:downsize//10]
val_df = val_df[:downsize//10]

In [ ]:
def preprocess_data(data):
    outs = []
    for doc in data['documents']:
        line = []
        line.append(doc['media_name'])
        line.append(doc['id'])
        para = []
        for sent in doc['text']:
            for s in sent:
                para.append(s['sentence'])
        line.append(para)
        line.append(doc['abstractive'][0])
        line.append(doc['extractive'])
        a = doc['extractive']
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)

    outs_df = pd.DataFrame(outs)
    outs_df.columns = ['media', 'id', 'article_original', 'abstractive', 'extractive']
    return outs_df

In [ ]:
train_df = preprocess_data(train_df)
train_df.head()

,media,id,article_original,abstractive,extractive
0,중도일보,353559055,"[금 2, 은 3, 동 2 획득, 올해 최고 성적인 종합 2위 달성, 제14회 대통...","서산시청 사격팀은 '제14회 대통령경호처장기 전국사격대회에서 금메달 2개, 은메달 ...","[2, 3, 4]"
1,전북일보,344076717,"[디지털뉴스팀, 펑키걸로 변신한 위키미키 / 사진 제공 = 판타지오, 위키미키(We...",위키미키가 지난 6일 공식sns채널을 통해 '잠겨있는 그들을 해방시킨다는' 뜻의 그...,"[2, 3, 4]"
2,머니투데이,357182579,"[베트남에 한국 정부 지원을 받은 농기계 임대사업 센터가 문을 열었다., 한국과 베...",22일(현지시각) 베트남 껀터에 한국정부지원을 받은 농기계 임대사업 센터가 문을 열...,"[0, 1, 2]"
3,부산일보,355443606,"[을숙도문화회관이 방학을 맞아 청소년을 위한 공연을 선보인다., 7일 프로무지카 부...",7일부터 을숙도문화회관이 방학을 맞은 청소년들이 부담없이 공연을 볼 수있도록 클래식...,"[0, 12, 3]"
4,남도일보,344827084,"[정세영 기자, 광주시, 수치지형도 수정 갱신 추진, 광산서부권·무등산권역 등 83...",광주광역시가 국토지리정보원과 함께 총 10억원을 투입해 장기간 갱신되지 않은 송산·...,"[3, 4, 5]"


In [ ]:
i = 4
print('===== 본    문 =====')
for idx, str in enumerate(train_df['article_original'][i]):
    print(idx,':',str)
print('===== 요약정답 =====')
print(train_df['extractive'][i])
print('===== 추출본문 =====')
print('1 :', train_df['article_original'][i][train_df['extractive'][i][0]])
print('2 :', train_df['article_original'][i][train_df['extractive'][i][1]])
print('3 :', train_df['article_original'][i][train_df['extractive'][i][2]])
print('===== 생성본문 =====')
print(train_df['abstractive'][i])

===== 본    문 =====
0 : 정세영 기자
1 : 광주시, 수치지형도 수정 갱신 추진
2 : 광산서부권·무등산권역 등 83㎢…도시 공간정보 제공
3 : 광주광역시가 관내 변화된 지형·지물 정보를 최신 정보로 수정하기 위해 1/1000 수치지형도 수정 갱신사업을 추진하고 있다.
4 : 사업은 국토지리정보원과 함께 총 10억원을 투입해 올해 12월까지 완료한다.
5 : 대상은 2003~2005년 최초 제작된 이후 장기간 갱신되지 않은 송산·삼도동 등 광산서부권역과 충효·청풍동 등 무등산권역 등 총 83㎢다.
6 : 수치지형도는 항공사진측량 기술로 지상의 위치와 지형, 건물, 도로 등 각종 공간정보를 일정한 축척에 따라 도형, 기호나 속성 등으로 표현한 전자지도다.
7 : 갱신된 내용은 국토정보플랫폼(http://map.ngii.go.kr)와 광주시 토지정보과(062-613-4592)에서 확인할 수 있다.
8 : 지형도에는 상·하수도와 가스, 통신관로 등 6대 지하시설물의 위치·속성 정보 등이 함께 제공돼 도시계획, 도로, 교통, 지역개발, 건축, 소방, 재난 등 시민생활과 밀접한 각종 행정시스템의 기초자료로 활용된다.
9 : 이수원 시 토지정보과장은 “도시 공간정보의 근간인 수치지형도 갱신을 지속적으로 추진해 최신 공간정보를 시민에게 제공하고 도시기반시설물 관리와 각종 공사 시행 시 기초자료 등으로 폭넓게 이용될 수 있도록 하겠다”고 말했다.
10 : /정세영 기자 jsy@namdonews.com
11 : 정세영 기자 jsy@namdonews.com
===== 요약정답 =====
[3, 4, 5]
===== 추출본문 =====
1 : 광주광역시가 관내 변화된 지형·지물 정보를 최신 정보로 수정하기 위해 1/1000 수치지형도 수정 갱신사업을 추진하고 있다.
2 : 사업은 국토지리정보원과 함께 총 10억원을 투입해 올해 12월까지 완료한다.
3 : 대상은 2003~2005년 최초 제작된 이후 장기간 갱신되지 않은 송산·삼도동 등 광산서부권역과 충효·청풍동

In [ ]:
i = 8
print('===== 본    문 =====')
for idx, str in enumerate(train_df['article_original'][i]):
    print(idx,':',str)
print('===== 요약정답 =====')
print(train_df['extractive'][i])
print('===== 추출본문 =====')
print('1 :', train_df['article_original'][i][train_df['extractive'][i][0]])
print('2 :', train_df['article_original'][i][train_df['extractive'][i][1]])
print('3 :', train_df['article_original'][i][train_df['extractive'][i][2]])
print('===== 생성본문 =====')
print(train_df['abstractive'][i])

===== 본    문 =====
0 : 특성화고 80명, 보훈대상자 20명
1 : 올 한해 총 750명 채용 계획
2 : 우리은행(은행장 손태승)은 일자리 창출과 우수 인재 확보를 위해 100여 명의 특별채용을 진행하고 있다고 23일 밝혔다.
3 : 우리은행은 이번 특별채용을 통해 특성화고 출신 80명, 국가보훈대상자 20명을 채용한다.
4 : 올해 특성화고 출신 채용 인원은 80명으로 이는 전년의 60명 대비 33% 증가한 규모다.
5 : 우리은행은 지난 8년간 금융권 최대 규모인 778명의 특성화고 출신 행원을 채용했다.
6 : 특성화고 출신 채용은 교육부와 협업을 통해 학교장 추천을 받은 고3 학생을 대상으로 진행되며, 현재 서류전형 합격자를 대상으로 1차 면접이 진행되고 있다.
7 : 올해 보훈 특별채용 인원은 총 40명으로 이는 전년 대비 100% 증가한 규모다.
8 : 우리은행은 지난 상반기에 국가보훈대상자 20명을 채용했으며, 하반기 역시 20명을 채용한다.
9 : 서류전형 합격자를 대상으로 23일 1차 면접을 진행 중이다.
10 : 보훈 특별채용과 관련 손태승 우리금융그룹회장 겸 우리은행장은 지난 7월 서울지방보훈청 주최로 진행된 '2019년 호국보훈의 달 대외 유공인사 국가보훈처장 감사패 전수식'에서 국가보훈처장 감사패를 받았다.
11 : 국가보훈처는 매년 6월 호국보훈의 달을 맞아 보훈대상자 취업에 모범이 되는 기관의 장에게 국가보훈처장감사패를 수여한다.
12 : 한편, 우리은행은 금융권 고용창출을 위해 올해 750명을 채용할 계획이다.
13 : 현재 상반기 300명 규모의 채용을 완료했으며, 하반기에는 본 특별채용을 포함하여 450명을 채용할 계획이다.
14 : 우리은행 채용 담당자는 "직무능력과 인성을 겸비한 신입행원을 선발하기 위하여 블라인드 면접을 진행하고 있다"며, "우리은행 취업을 위해 부단히 준비해 온 지원자에게 좋은 결과가 있기를 바란다"고 말했다.
===== 요약정답 =====
[2, 12, 14]
===== 추출본문 ===

In [ ]:
train_df.head(1)

,media,id,article_original,abstractive,extractive
0,중도일보,353559055,"[금 2, 은 3, 동 2 획득, 올해 최고 성적인 종합 2위 달성, 제14회 대통...","서산시청 사격팀은 '제14회 대통령경호처장기 전국사격대회에서 금메달 2개, 은메달 ...","[2, 3, 4]"


In [ ]:
test_df = preprocess_data(test_df)
test_df.head(1)

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"


In [ ]:
val_df = preprocess_data(val_df)
val_df.head(1)

,media,id,article_original,abstractive,extractive
0,전북일보,351014195,"[최명국, 송하진 지사, 방중 주요 성과로 '군산~연운항' 항로 개설 협의 꼽아, ...",송하진 전북도지사가 중국 장쑤성을 방문해 러우 친지앤 당서기와 새만금 산단 5공구 ...,"[3, 7, 8]"


## 토크나이저

KoBART
https://huggingface.co/gogamza/kobart-base-v1

In [ ]:
from transformers import PreTrainedTokenizerFast, BartModel

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
model = BartModel.from_pretrained('gogamza/kobart-base-v1')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
class SummDataset(Dataset):

    def __init__(
        self, 
        data: pd.DataFrame, 
        tokenizer: PreTrainedTokenizerFast, 
        max_token_len: int = 512
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        tokenlist = []
        for sent in data_row.article_original:
            tokenlist.append(tokenizer(
                text = sent,
                add_special_tokens = True)) #, # Add '[CLS]' and '[SEP]'
    
        src = [] # 토크나이징 된 전체 문단
        labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
        segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
        clss = []  #[CLS]토큰의 포지션값을 지정

        odd = 0
        for tkns in tokenlist:
            if odd > 1 : odd = 0
            clss = clss + [len(src)]
            src = src + tkns['input_ids']
            segs = segs + [odd] * len(tkns['input_ids'])
            if tokenlist.index(tkns) in data_row.extractive :
                labels = labels + [1]
            else:
                labels = labels + [0]
            odd += 1
        
            #truncation
            if len(src) == MAX_TOKEN_COUNT:
                break
            elif len(src) > MAX_TOKEN_COUNT:
                src = src[:self.max_token_len - 1] + [src[-1]]
                segs = segs[:self.max_token_len]
                break
    
        #padding
        if len(src) < MAX_TOKEN_COUNT:
            src = src + [0]*(self.max_token_len - len(src))
            segs = segs + [0]*(self.max_token_len - len(segs))
            
        if len(clss) < MAX_TOKEN_COUNT:
            clss = clss + [-1]*(self.max_token_len - len(clss))
        if len(labels) < MAX_TOKEN_COUNT:
            labels = labels + [0]*(self.max_token_len - len(labels))

        return dict(
            src = torch.tensor(src),
            segs = torch.tensor(segs),
            clss = torch.tensor(clss),
            labels= torch.FloatTensor(labels)
        )

In [ ]:
class SummDataModule(pl.LightningDataModule):

    def __init__(self, train_df, test_df, val_df, tokenizer, batch_size=1, max_token_len=512):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.val_df = val_df
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = SummDataset(
            self.train_df,
            self.tokenizer,
            self.max_token_len
        )

        self.test_dataset = SummDataset(
            self.test_df,
            self.tokenizer,
            self.max_token_len
        )
    
        self.val_dataset = SummDataset(
            self.val_df,
            self.tokenizer,
            self.max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def test_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

In [ ]:
data_module = SummDataModule(
  train_df,
  test_df,  
  val_df,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_TOKEN_COUNT
)

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, dropout, dim, max_len=5000):
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, dim, 2, dtype=torch.float) *
                              -(math.log(10000.0) / dim)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)
        pe = pe.unsqueeze(0)
        super(PositionalEncoding, self).__init__()
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
        self.dim = dim

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim)
        if (step):
            emb = emb + self.pe[:, step][:, None, :]

        else:
            emb = emb + self.pe[:, :emb.size(1)]
        emb = self.dropout(emb)
        return emb

    def get_emb(self, emb):
        return self.pe[:, :emb.size(1)]

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attn = MultiHeadedAttention(
            heads, d_model, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout) 

    def forward(self, iter, query, inputs, mask):
        if (iter != 0):
            input_norm = self.layer_norm(inputs)
        else:
            input_norm = inputs

        mask = mask.unsqueeze(1)
        context = self.self_attn(input_norm, input_norm, input_norm,
                                 mask=mask)
        out = self.dropout(context) + inputs
        return self.feed_forward(out)

In [ ]:
class ExtTransformerEncoder(nn.Module):
    def __init__(self, hidden_size=768, d_ff=2048, heads=8, dropout=0.3, num_inter_layers=2): 
        super(ExtTransformerEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_inter_layers = num_inter_layers
        self.pos_emb = PositionalEncoding(dropout, hidden_size)
        self.transformer_inter = nn.ModuleList(
            [TransformerEncoderLayer(hidden_size, heads, d_ff, dropout)
            for _ in range(num_inter_layers)])
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_size, eps=1e-6)
        self.wo = nn.Linear(hidden_size, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, top_vecs, mask):
        """ See :obj:`EncoderBase.forward()`"""

        batch_size, n_sents = top_vecs.size(0), top_vecs.size(1)
        pos_emb = self.pos_emb.pe[:, :n_sents]
        x = top_vecs * mask[:, :, None].float()
        x = x + pos_emb

        for i in range(self.num_inter_layers):
            x = self.transformer_inter[i](i, x, x, ~mask) 

        x = self.layer_norm(x)
        sent_scores = self.sigmoid(self.wo(x))
        sent_scores = sent_scores.squeeze(-1) * mask.float()

        return sent_scores

In [ ]:
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def gelu(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


    def forward(self, x):
        inter = self.dropout_1(self.gelu(self.w_1(self.layer_norm(x))))
        output = self.dropout_2(self.w_2(inter))
        return output + x

In [ ]:
class MultiHeadedAttention(nn.Module):

    def __init__(self, head_count, model_dim, dropout=0.1, use_final_linear=True):
        assert model_dim % head_count == 0
        self.dim_per_head = model_dim // head_count
        self.model_dim = model_dim

        super(MultiHeadedAttention, self).__init__()
        self.head_count = head_count

        self.linear_keys = nn.Linear(model_dim,
                                     head_count * self.dim_per_head)
        self.linear_values = nn.Linear(model_dim,
                                       head_count * self.dim_per_head)
        self.linear_query = nn.Linear(model_dim,
                                      head_count * self.dim_per_head)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        self.use_final_linear = use_final_linear
        if (self.use_final_linear):
            self.final_linear = nn.Linear(model_dim, model_dim)
    def forward(self, key, value, query, mask=None,
                layer_cache=None, type=None, predefined_graph_1=None):

        batch_size = key.size(0)
        dim_per_head = self.dim_per_head
        head_count = self.head_count
        key_len = key.size(1)
        query_len = query.size(1)

        def shape(x):
            """  projection """
            return x.view(batch_size, -1, head_count, dim_per_head) \
                .transpose(1, 2)
        def unshape(x):
            """  compute context """
            return x.transpose(1, 2).contiguous() \
                .view(batch_size, -1, head_count * dim_per_head)

        # 1) Project key, value, and query.
        if layer_cache is not None:
            if type == "self":
                query, key, value = self.linear_query(query), \
                                    self.linear_keys(query), \
                                    self.linear_values(query)

                key = shape(key)
                value = shape(value)

                if layer_cache is not None:
                    device = key.device
                    if layer_cache["self_keys"] is not None:
                        key = torch.cat(
                            (layer_cache["self_keys"].to(device), key),
                            dim=2)
                    if layer_cache["self_values"] is not None:
                        value = torch.cat(
                            (layer_cache["self_values"].to(device), value),
                            dim=2)
                    layer_cache["self_keys"] = key
                    layer_cache["self_values"] = value
            elif type == "context":
                query = self.linear_query(query)
                if layer_cache is not None:
                    if layer_cache["memory_keys"] is None:
                        key, value = self.linear_keys(key), \
                                     self.linear_values(value)
                        key = shape(key)
                        value = shape(value)
                    else:
                        key, value = layer_cache["memory_keys"], \
                                     layer_cache["memory_values"]
                    layer_cache["memory_keys"] = key
                    layer_cache["memory_values"] = value
                else:
                    key, value = self.linear_keys(key), \
                                 self.linear_values(value)
                    key = shape(key)
                    value = shape(value)
        else:
            key = self.linear_keys(key)
            value = self.linear_values(value)
            query = self.linear_query(query)
            key = shape(key)
            value = shape(value)

        query = shape(query)

        key_len = key.size(2)
        query_len = query.size(2)

        # 2) Calculate and scale scores.
        query = query / math.sqrt(dim_per_head)
        scores = torch.matmul(query, key.transpose(2, 3))

        if mask is not None:
            mask = mask.unsqueeze(1).expand_as(scores)
            scores = scores.masked_fill(mask, -1e18) # how can i fix it to use fp16...

        # 3) Apply attention dropout and compute context vectors.

        attn = self.softmax(scores)

        if (not predefined_graph_1 is None):
            attn_masked = attn[:, -1] * predefined_graph_1
            attn_masked = attn_masked / (torch.sum(attn_masked, 2).unsqueeze(2) + 1e-9)

            attn = torch.cat([attn[:, :-1], attn_masked.unsqueeze(1)], 1)

        drop_attn = self.dropout(attn)
        if (self.use_final_linear):
            context = unshape(torch.matmul(drop_attn, value))
            output = self.final_linear(context)
            return output
        else:
            context = torch.matmul(drop_attn, value)
            return context

In [ ]:
class Summarizer(pl.LightningModule):

    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.max_pos = 512
        self.bert = BertModel.from_pretrained('gogamza/kobart-base-v1') #, return_dict=True)
        self.ext_layer = ExtTransformerEncoder()
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.loss = nn.BCELoss(reduction='none')
    
        for p in self.ext_layer.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

    def forward(self, src, segs, clss, labels=None): #, input_ids, attention_mask, labels=None):
        
        mask_src = ~(src == 0) #1 - (src == 0)
        mask_cls = ~(clss == -1) #1 - (clss == -1)

        top_vec = self.bert(src, token_type_ids=segs, attention_mask=mask_src)
        top_vec = top_vec.last_hidden_state
        
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()

        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)
        
        loss = 0
        if labels is not None:
            loss = self.loss(sent_scores, labels)
            
            loss = (loss * mask_cls.float()).sum() / len(labels)
        
        return loss, sent_scores

    def step(self, batch):

        src = batch['src']
        if len(batch['labels']) > 0 :
            labels = batch['labels']
        else:
            labels = None
        segs = batch['segs']
        clss = batch['clss']
        
        loss, sent_scores = self(src, segs, clss, labels)    
        
        return loss, sent_scores, labels

    def training_step(self, batch, batch_idx):

        loss, sent_scores, labels = self.step(batch)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def validation_step(self, batch, batch_idx):
        
        loss, sent_scores, labels = self.step(batch)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def test_step(self, batch, batch_idx):
        
        loss, sent_scores, labels = self.step(batch)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def acc_loss(self, outputs):
        total_loss = 0
        hit_cnt = 0
        for outp in outputs:
            labels = outp['labels'].cpu()
            predictions, idxs = outp['predictions'].cpu().sort()
            loss = outp['loss'].cpu()
            for label, idx in zip(labels, idxs):
                for i in range(1,3):
                    if label[idx[-i-1]] == 1 : 
                        hit_cnt += 1

            total_loss += loss
            
        avg_loss = total_loss / len(outputs)
        acc = hit_cnt / (3*len(outputs)*len(labels))
        
        return acc, avg_loss

    def training_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('acc:', acc, 'avg_loss:', avg_loss)
        
        self.log('avg_train_loss', avg_loss, prog_bar=True, logger=True)

    def validation_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('val_acc:', acc, 'avg_val_loss:', avg_loss)
        
        self.log('avg_val_loss', avg_loss, prog_bar=True, logger=True)

    def test_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('test_acc:', acc, 'avg_test_loss:', avg_loss)
        
        self.log('avg_test_loss', avg_loss, prog_bar=True, logger=True)

        return

    def configure_optimizers(self):
        
        optimizer = AdamW(self.parameters(), lr=2e-5)

        steps_per_epoch=len(train_df) // BATCH_SIZE
        total_training_steps = steps_per_epoch * N_EPOCHS
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=steps_per_epoch,
            num_training_steps=total_training_steps
        )

        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
            )
        )

In [ ]:
model = Summarizer()

## 학습

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="avg_val_loss",
    mode="min"
)

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="kobart_Summary")

In [ ]:
early_stopping_callback = EarlyStopping(monitor='avg_val_loss', patience=3)

https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html#pytorch_lightning.trainer.Trainer.params.gpus

In [ ]:
trainer = pl.Trainer(
    callbacks=[checkpoint_callback,early_stopping_callback],
    logger=logger,
    max_epochs=N_EPOCHS, # 2
    gpus=1,
)

In [ ]:
trainer.fit(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                  | Params
----------------------------------------------------
0 | bert      | BertModel             | 66.9 M
1 | ext_layer | ExtTransformerEncoder | 11.0 M
2 | loss      | BCELoss               | 0     
----------------------------------------------------
78.0 M    Trainable params
0         Non-trainable params
78.0 M    Total params
311.917   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

val_acc: 0.25 avg_val_loss: tensor(8.4448)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

val_acc: 0.31133333333333335 avg_val_loss: tensor(6.0769)
acc: 0.31333333333333335 avg_loss: tensor(5.8929)


INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 10000: 'avg_val_loss' reached 6.07686 (best 6.07686), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

val_acc: 0.309 avg_val_loss: tensor(5.9282)
acc: 0.3409333333333333 avg_loss: tensor(5.4879)


INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 15000: 'avg_val_loss' reached 5.92817 (best 5.92817), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


## 요약 결과

In [ ]:
trained_model = Summarizer.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)
trained_model.eval()
trained_model.freeze()

In [ ]:
torch.save(trained_model, '/content/drive/MyDrive/데이터/금융 요약/model_fulldata2.pt')

In [ ]:
trained_model = torch.load('/content/drive/MyDrive/데이터/금융 요약/model_fulldata2.pt') 

In [ ]:
!pip install python-mecab-kor

In [ ]:
import pandas as pd
import numpy as np
news = pd.read_csv("/content/drive/MyDrive/데이터/금융 요약/기사 예시.csv")
news.head()

,제목,본문
0,당분간 숨고르기 이어질 듯... 잭슨홀 미팅·금통위 '변수',이번주 코스피지수는 주요국 중앙은행 총재들이 모이는 잭슨홀 미팅과 한국은행 금융통화...
1,"물가 피크아웃 기대에 약세장 탈피…""횡보장 전망""",코스피 예상범위 2450~2550선코스피지수는 이번 주 2550포인트와 2450포인...
2,美 잭슨홀 미팅·한은 금통위 주목…2500선 안착 시험대,미국 연방준비제도(Fed·연준)의 7월 연방공개시장위원회(FOMC) 의사록에서 금리...
3,"""3분기도 곡물가격 오른다""…사료주 강세",올해 3분기에도 곡물 수입 가격이 오를 것으로 예상되면서 최근 사료 기업 주가가 상...
4,지정학적 위기가 기회로… 날아오르는 방산株,최근 글로벌 금융시장에 지정학적 위기가 고조되며 방산주의 상승세가 돋보이고 있다. ...


In [ ]:
sentence = []
for news_sentence in news['본문'].values:
    sentence.append(news_sentence)

In [ ]:
def data_process(text):
    # 문장 분리 하고,
    sents = kss.split_sentences(text)
    
    #데이터 가공하고,
    tokenlist = []
    for sent in sents:
        tokenlist.append(tokenizer(
            text = sent,
            add_special_tokens = True)) #, # Add '[CLS]' and '[SEP]'

    src = [] # 토크나이징 된 전체 문단
    labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
    segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
    clss = []  #[CLS]토큰의 포지션값을 지정

    odd = 0

    for tkns in tokenlist:

        if odd > 1 : odd = 0
        clss = clss + [len(src)]
        src = src + tkns['input_ids']
        segs = segs + [odd] * len(tkns['input_ids'])
        odd += 1

        #truncation
        if len(src) == MAX_TOKEN_COUNT:
            break
        elif len(src) > MAX_TOKEN_COUNT:
            src = src[:MAX_TOKEN_COUNT - 1] + [src[-1]]
            segs = segs[:MAX_TOKEN_COUNT]
            break

    #padding
    if len(src) < MAX_TOKEN_COUNT:
        src = src + [0]*(MAX_TOKEN_COUNT - len(src))
        segs = segs + [0]*(MAX_TOKEN_COUNT - len(segs))

    if len(clss) < MAX_TOKEN_COUNT:
        clss = clss + [-1]*(MAX_TOKEN_COUNT - len(clss))

    return dict(
        sents = sents, #정답 출력을 위해...
        src = torch.tensor(src),
        segs = torch.tensor(segs),
        clss = torch.tensor(clss),
    )

In [ ]:
def summarize_test(text):
    data = data_process(text.replace('\n',''))
    
    #trained_model에 넣어 결과값 반환
    _, rtn = trained_model(data['src'].unsqueeze(0), data['segs'].unsqueeze(0), data['clss'].unsqueeze(0))
    rtn = rtn.squeeze()
    
    # 예측 결과값을 받기 위한 프로세스
    rtn_sort, idx = rtn.sort(descending = True)
    
    rtn_sort = rtn_sort.tolist()
    idx = idx.tolist()

    end_idx = rtn_sort.index(0)

    rtn_sort = rtn_sort[:end_idx]
    idx = idx[:end_idx]
    
    if len(idx) > 3:
        rslt = idx[:3]
    else:
        rslt = idx
        
    summ = []
    print(' ')
    for i, r in enumerate(rslt):
        summ.append(data['sents'][r])
        print(summ[i])

    return summ

In [ ]:
for i in sentence:
  summarize_test(i)

 
이번주 코스피지수는 주요국 중앙은행 총재들이 모이는 잭슨홀 미팅과 한국은행 금융통화위원회(금통위)를 앞두고 숨고르기 장세가 이어질 것으로 전망된다.
증시 전문가들은 이번주 코스피 예상밴드를 2450~2550으로 제시했다.
코스닥지수는 2.1% 하락한 814.17에 장을 마감했다.


 
코스피 예상범위 2450~2550선코스피지수는 이번 주 2550포인트와 2450포인트를 상·하단으로 오가는 제한적인 흐름을 보일 것이란 전망이 나온다.
21일 금융투자업계에 따르면 이번 주 국내 증시의 주요 상승과 하락 요인은 물가 피크아웃 기대감과 기업들의 이익 하향 조정 등이 거론된다.
인플레이션 피크아웃에 대한 기대감이 형성되면서 최근 코스피지수는 2500선을 회복했지만, 미국 연방준비제도(Fed·연준)의 금리 인상 경로가 여전히 불확실한 상태이기 때문이다.


 
미국 연방준비제도(Fed·연준)의 7월 연방공개시장위원회(FOMC) 의사록에서 금리 인상 의지가 재확인되면서 코스피 지수가 2490선으로 미끄러졌다.
참석자들이 인플레이션이 잡힐 때까지 경제 성장세를 꺾을 정도로 기준금리를 올려야 한다고 강조하면서 긴축 우려가 거듭 부각된 탓이다.
이번주(22~26일)에는 주요국 중앙은행 총재들이 모이는 잭슨홀 미팅과 한국은행 금융통화의원회(금통위)가 열리는 만큼 숨고르기 장세에 들어갈 것으로 전망된다.


 
21일 한국거래소에 따르면 고려산업 주가는 지난주(16~19일)에 전주(12일) 대비 22.5% 상승한 6070원으로 장을 마감했다.
지난 17일에는 전 일 대비 18.20%, 18일에는 5.41% 급등했다.
올해 3분기에도 곡물 수입 가격이 오를 것으로 예상되면서 최근 사료 기업 주가가 상승세를 보이고 있다.
 
최근 글로벌 금융시장에 지정학적 위기가 고조되며 방산주의 상승세가 돋보이고 있다.
21일 한국거래소에 따르면 한화에어로스페이스는 지난 16일 장중 7만4000원에 거래되며 지난달 13일(4만850원) 이후 한 달 만에 81.15% 급등했다.
한국 방산업체들의 이른바 '가성비'가 높게 점쳐지면서 정부 차원의 세일즈 외교가 큰 성과를 보일 수 있다는 긍정론도 나온다.
